In [2]:
# for google colab
from google.colab import drive
# mount your Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# for google colab
# copy all files from "project1" directory in Google drive to current directory
!cp -r ./gdrive/MyDrive/project1/* .

In [4]:
from tensorflow import keras
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import tree
from sklearn.svm import SVC
from keras.utils import to_categorical
import keras
from keras.models import Sequential, Model 
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Load Data
using keras datsbase to import Fashion-MNIST dataset. 


In [8]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
# used when testing the performance of smaller training dataset
X_train, X_re, y_train, y_re = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [6]:
# normalization
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# reshape the data size to fit the classifier
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2]) 
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

# PCA
Doing PCA on dataset to reduce the dimension

In [6]:
pca = PCA(n_components=0.85)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

# KNN

In [7]:
# loop from 1 to 10 to get the best k
for k in range(1, 11):
    # train the data with kNN classifier
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train) 
    # test data
    y_pred = model.predict(X_test) 

    # evaluate the model with accuracy, precision, recall, f1-score, aucroc
    print("k=%d, accuracy=%.2f%%" % (k, metrics.accuracy_score(y_test, y_pred) * 100))
    print("k=%d, precision=%.2f%%" % (k, metrics.precision_score(y_test, y_pred, average='weighted') * 100))
    print("k=%d, recall=%.2f%%" % (k, metrics.recall_score(y_test, y_pred, average='weighted') * 100))
    print("k=%d, f1_score=%.2f%%" % (k, metrics.f1_score(y_test, y_pred, average="weighted") * 100))
    print("k=%d, AUROC=%.2f%%" % (k, metrics.roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr') * 100))
    print('\n')

k=5, accuracy=85.06%
k=5, precision=85.10%
k=5, recall=85.06%
k=5, f1_score=85.00%
k=5, AUROC=96.83%




# SVM

In [11]:
# using SVM with different kernel (Gaussian Kernel, Linear Kernel, Polynomial Kernel) to predict Fashion-MNIST
# Gaussian Kernel
model = SVC(kernel='rbf', decision_function_shape='ovr', probability=True)
# Linear Kernel
# model = SVC(kernel='linear', decision_function_shape='ovr', probability=True)
# Polynomial Kernel
# model = SVC(C=10, kernel='poly', gamma="auto", probability=True)
model.fit(X_train, y_train)
# predcting the data
y_pred = model.predict(X_test)

# evaluate the model with accuracy, precision, recall, f1-score, aucroc
print("accuracy=%.2f%%" % (metrics.accuracy_score(y_test, y_pred) * 100))
print("precision=%.2f%%" % (metrics.precision_score(y_test, y_pred, average='weighted') * 100))
print("recall=%.2f%%" % (metrics.recall_score(y_test, y_pred, average='weighted') * 100))
print("f1_score=%.2f%%" % (metrics.f1_score(y_test, y_pred, average="weighted") * 100))
print("AUROC=%.2f%%" % (metrics.roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr') * 100))

Gaussian Kernel
accuracy=86.96%
precision=86.93%
recall=86.96%
f1_score=86.93%
AUROC=98.87%


# Random Forest

In [9]:
# using random forest with different n_estimators to predict data 
estimators = [100, 300, 1000]

for n in estimators:
    # training the random forest classifier
    model = RandomForestClassifier(n_estimators=n, max_depth=100, random_state=42)
    model = model.fit(X_train, y_train)
    # predciting data
    y_pred = model.predict(X_test)

    # evaluate the model with accuracy, precision, recall, f1-score, aucroc
    print("accuracy=%.2f%%" % (metrics.accuracy_score(y_test, y_pred) * 100))
    print("precision=%.2f%%" % (metrics.precision_score(y_test, y_pred, average='weighted') * 100))
    print("recall=%.2f%%" % (metrics.recall_score(y_test, y_pred, average='weighted') * 100))
    print("f1_score=%.2f%%" % (metrics.f1_score(y_test, y_pred, average="weighted") * 100))
    print("AUROC=%.2f%%" % (metrics.roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr') * 100))

accuracy=86.24%
precision=86.12%
recall=86.24%
f1_score=86.13%
AUROC=98.79%


# Convolutional Neural Network

In [7]:
# reshape data to fit the input of CNN model
X_train= X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape, X_test.shape
# do one-hot-encoding
y_train = to_categorical(y_train)

# For latter use to evaluate the CNN model
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

(48000, 784)


In [ ]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

datagen.fit(X_train)

In [ ]:
def build_model(classes=10, optimizer='adam'):
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu' ,padding='same', input_shape=(28, 28, 1)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu' ,padding='same', input_shape=(28, 28, 1)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu' ,padding='same', input_shape=(28, 28, 1)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu' ,padding='same', input_shape=(28, 28, 1)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=256, kernel_size=(3, 3),activation='relu',padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(256,activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(classes, activation='softmax'))
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
# build model
model = build_model()

# Training / Testing

In [ ]:
# training the CNN model
model.fit(X_train, y_train, batch_size=64, epochs=25, verbose=1, validation_data=(X_val, y_val))
# training the CNN model with data augmentation
# model.fit(datagen.flow(X_train, y_train, batch_size=64), batch_size=64, epochs=25, verbose=1, validation_data=(X_val, y_val))

In [ ]:
# predicting the data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis = 1) 

# evaluate the model with accuracy, precision, recall, f1-score, aucroc
print("accuracy=%.2f%%" % (metrics.accuracy_score(y_test, y_pred_classes) * 100))
print("precision=%.2f%%" % (metrics.precision_score(y_test, y_pred_classes, average='weighted') * 100))
print("recall=%.2f%%" % (metrics.recall_score(y_test, y_pred_classes, average='weighted') * 100))
print("f1_score=%.2f%%" % (metrics.f1_score(y_test, y_pred_classes, average="weighted") * 100))
print("AUROC=%.2f%%" % (metrics.roc_auc_score(y_test, y_pred, multi_class='ovr') * 100))